In [4]:
import numpy as np
from scipy.optimize import curve_fit    # フィッティング用
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.models import load_model
import os
from keras import layers
from keras import models
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop
import tensorflow as tf
import time
import csv

os.environ['KMP_DUPLICATE_LIB_OK']='TRUE' #意味はわからん

In [10]:
start = time.time()
train_num = 10000 #訓練データの数
test_num = 1000 #テストデータの数
data_size = 100 #配列の大きさ

x_train = np.zeros((train_num,data_size))
y_train = np.zeros((train_num,data_size))
tx_train = np.zeros((train_num, 3))
ty_train = np.zeros((train_num, 3))
t_1_train = np.zeros(train_num)
x_test = np.zeros((test_num,data_size))
y_test = np.zeros((test_num,data_size))
tx_test = np.zeros((test_num, 3))
ty_test = np.zeros((test_num, 3))
t_1_test = np.zeros(test_num)

# ガウシアンビームの関数の定義
def gaussian_beam(x,a,b,c,d):
    return  a * np.exp(-2*(x-b)*(x-b)/c/c) + d

# ガウシアンビームのパラメータ
i0 = 1.0
x0 = 0.0
y0 = 0.0
w0 = 5.0
h0 = 0.0
param_ini_x = np.array([i0, x0, w0, h0])  # フィッティングの初期値 (ここではデータから推定は行わない)
param_ini_y = np.array([i0, y0, w0, h0])

# 信号に対するノイズ量[%]
NOISE = 30

print(x0, y0)
def create_data(N, n_N, x0_N, y0_N, w0_x_N, w0_y_N, h0_x_N, h0_y_N, i0, x0, y0, w0, h0, x, y, tx, ty, t1):
    n_N      = np.zeros(N)
    x0_N     = np.zeros(N)   # N回分のx0,w0を格納する配列
    y0_N     = np.zeros(N)
    w0_x_N   = np.zeros(N)
    w0_y_N   = np.zeros(N)
    h0_x_N   = np.zeros(N)
    h0_y_N   = np.zeros(N)
    
    k = 1

    for n in range(N):        # 繰り返し精度を調べるために各ノイズ割合でN回実行

        n_N[n] = n + 1
        
        center = data_size/2
        # x配列とy配列
        x_array = np.arange(-center, center, 1.0)                         # x配列
        y_array = np.arange(-center, center, 1.0)                         # y配列
        nx = len(x_array)
        ny = len(y_array)
        intensity = np.zeros((nx, ny))                            # ノイズを含まない2次元強度分布
        x0 = x0 + center
        y0 = y0 + center
        for i in range(nx):
            for j in range(ny):
                intensity[i][j] = i0 * np.exp(-2*((i-x0)*(i-x0) + (j-y0)*(j-y0))/w0/w0)
        x0 = x0 - center
        y0 = y0 - center
        # 最大強度を取る位置における強度プロファイル
        profile_x = np.zeros(nx)
        profile_y = np.zeros(ny)
    

        # 2次元の強度分布にノイズを付与
        noise = (np.random.rand(nx*ny)-0.5)*i0*NOISE*0.01   #プラスマイナスNOISE%のノイズ(一様分布), (np.random.rand(nx*ny)-0.5)*2の部分が-1から1までの乱数になる
        noise = noise.reshape((nx,ny))
        intensity_noise = intensity + noise

        # 最大値の探索 & その位置の強度プロファイルの取得
        idx = np.unravel_index(np.argmax(intensity_noise), intensity_noise.shape)
        x[n]= intensity_noise[:,idx[1]]
        profile_y = intensity_noise[idx[0],:]
        y[n] = profile_y
        tx[n] = (i0, x0, w0)
        ty[n] = (i0, y0, w0)

        # x方向のプロファイルの非線形フィッティング
        param, cov  = curve_fit(gaussian_beam, x_array, profile_x, p0=param_ini_x, maxfev=2000)
        i0_x        = param[0]
        x0_N[n]     = param[1]
        w0_x_N[n]   = param[2]
        h0_x_N[n]   = param[3]
        # y方向のプロファイルの非線形フィッティング
        param, cov  = curve_fit(gaussian_beam, y_array, profile_y, p0=param_ini_y, maxfev=2000)
        i0_y        = param[0]
        y0_N[n]     = param[1]
        w0_y_N[n]   = param[2]
        h0_y_N[n]   = param[3]
        
        #中心位置の変化
        i = N*k / 20
        if n > i:
            x0 = x0 + 0.1
            y0 = y0 + 0.1
            k = k+1
            print(x0, y0)
        elif n == N-1:
            print(x0, y0)
        else:
            x0 = x0
    return N, n_N, x0_N, y0_N, w0_x_N, w0_y_N, h0_x_N, h0_y_N, x, y, tx, ty

# N回繰り返しフィッティングを行う
N        = 10000
n_N      = np.zeros(N)
x0_N     = np.zeros(N)   # N回分のx0,w0を格納する配列
y0_N     = np.zeros(N)
w0_x_N   = np.zeros(N)
w0_y_N   = np.zeros(N)
h0_x_N   = np.zeros(N)
h0_y_N   = np.zeros(N)

create_data(train_num, n_N, x0_N, y0_N, w0_x_N, w0_y_N, h0_x_N, h0_y_N, 
            i0, x0, y0, w0, h0, x_train, y_train, tx_train, ty_train, t_1_train)

# N回繰り返しフィッティングを行う
N        = 1000
n_N      = np.zeros(N)
x0_N     = np.zeros(N)   # N回分のx0,w0を格納する配列
y0_N     = np.zeros(N)
w0_x_N   = np.zeros(N)
w0_y_N   = np.zeros(N)
h0_x_N   = np.zeros(N)
h0_y_N   = np.zeros(N)

create_data(test_num, n_N, x0_N, y0_N, w0_x_N, w0_y_N, h0_x_N, h0_y_N, 
            i0, x0, y0, w0, h0, x_test, y_test, tx_test, ty_test, t_1_test)

elapsed_time = time.time() - start
print ("elapsed_time:{0}".format(elapsed_time) + "[sec]")

0.0 0.0
0.1 0.1
0.20000000000000143 0.20000000000000143
0.3000000000000028 0.3000000000000028
0.40000000000000424 0.40000000000000424
0.5000000000000057 0.5000000000000057
0.6000000000000071 0.6000000000000071
0.7000000000000085 0.7000000000000085
0.8000000000000099 0.8000000000000099
0.9000000000000113 0.9000000000000113
1.0000000000000129 1.0000000000000129
1.1000000000000143 1.1000000000000143
1.2000000000000157 1.2000000000000157
1.3000000000000171 1.3000000000000171
1.4000000000000186 1.4000000000000186
1.50000000000002 1.50000000000002
1.6000000000000214 1.6000000000000214
1.7000000000000228 1.7000000000000228
1.8000000000000242 1.8000000000000242
1.9000000000000257 1.9000000000000257
1.900000000000027 1.900000000000027
0.1 0.1
0.20000000000000143 0.20000000000000143
0.3000000000000028 0.3000000000000028
0.40000000000000424 0.40000000000000424
0.5000000000000057 0.5000000000000057
0.6000000000000071 0.6000000000000071
0.7000000000000085 0.7000000000000085
0.8000000000000099 0.800

In [16]:
t_train = tx_train
t_test = tx_test

#訓練データ,テストデータの保存
save_x_train = "/Users/nagaiyuma/Desktop/maindata/x_train.csv"
save_t_train = "/Users/nagaiyuma/Desktop/maindata/t_train.csv"
save_x_test = "/Users/nagaiyuma/Desktop/maindata/x_test.csv"
save_t_test = "/Users/nagaiyuma/Desktop/maindata/t_test.csv"

with open(save_x_train,'w') as f:
    writer = csv.writer(f)
    writer.writerows(x_train)
with open(save_t_train,'w') as f:
    writer = csv.writer(f)
    writer.writerows(t_train)
with open(save_x_test,'w') as f:
    writer = csv.writer(f)
    writer.writerows(x_test)
with open(save_t_test,'w') as f:
    writer = csv.writer(f)
    writer.writerows(t_test)

In [15]:
print(tx_test)

[[1.  0.  5. ]
 [1.  0.  5. ]
 [1.  0.  5. ]
 ...
 [1.  1.9 5. ]
 [1.  1.9 5. ]
 [1.  1.9 5. ]]


In [6]:
#CNN3の実装
batch_size = 200  # 訓練データを200ずつのデータに分けて学習させる
epochs = 50 # 訓練データを繰り返し学習させる数

#データ形式の変更(シーケンス長, パラメータ数)
train_x = x_train.reshape(-1, 100, 1)
train_t = tx_train
test_x = x_test.reshape(-1, 100, 1)
test_t = tx_test

model = Sequential()
model.add(Conv1D(50, 3, padding='same', input_shape=(100, 1), activation='relu', kernel_initializer="he_normal"))
model.add(MaxPooling1D(2, padding='same'))
model.add(Conv1D(50, 3, padding='same', activation='relu', kernel_initializer="he_normal"))
model.add(MaxPooling1D(5, padding='same'))
model.add(Conv1D(10, 3, padding='same', activation='relu', kernel_initializer="he_normal"))
model.add(layers.Flatten())
model.add(Dense(100, activation='relu', kernel_initializer="he_normal"))
model.add(Dropout(0.2))
model.add(Dense(10, activation='relu', kernel_initializer="he_normal"))
model.add(Dropout(0.2))
model.add(Dense(3, activation='relu', kernel_initializer="he_normal"))

model.summary()

model.compile(loss='mse',
 optimizer='adam',
 metrics=['mae'])


callbacks = [tf.keras.callbacks.TensorBoard(log_dir='my_log_dir', 
                                         histogram_freq=1, 
                                         embeddings_freq=1)]

history = model.fit(train_x, train_t,
 batch_size=batch_size,
 epochs=epochs,
 verbose=1,
 validation_data=(test_x, test_t),
 callbacks=callbacks)

score = model.evaluate(test_x, test_t, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_10 (Conv1D)           (None, 100, 50)           200       
_________________________________________________________________
max_pooling1d_7 (MaxPooling1 (None, 50, 50)            0         
_________________________________________________________________
conv1d_11 (Conv1D)           (None, 50, 50)            7550      
_________________________________________________________________
max_pooling1d_8 (MaxPooling1 (None, 10, 50)            0         
_________________________________________________________________
conv1d_12 (Conv1D)           (None, 10, 10)            1510      
_________________________________________________________________
flatten_4 (Flatten)          (None, 100)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 100)              

InvalidArgumentError: You must feed a value for placeholder tensor 'conv1d_4_input' with dtype float and shape [?,100,1]
	 [[{{node conv1d_4_input}}]]

In [ ]:
#modelの保存
save_model_path = "/Users/nagaiyuma/Desktop/maindata/model_cnn_3.h5"
model.save(save_model_path)

In [ ]:
#学習パラメータの取得
model = load_model(save_model_path)

In [ ]:
abs_error = abs(np.array(model.predict(test_x).reshape(1000,3)).T[0]- tx_test.T[0])
ave_abs = np.average(abs_error)
std_abs = np.std(abs_error)
print(ave_abs, std_abs)
abs_error = abs(np.array(model.predict(test_x).reshape(1000,3)).T[1]- tx_test.T[1])
ave_abs = np.average(abs_error)
std_abs = np.std(abs_error)
print(ave_abs, std_abs)
abs_error = abs(np.array(model.predict(test_x).reshape(1000,3)).T[2]- tx_test.T[2])
ave_abs = np.average(abs_error)
std_abs = np.std(abs_error)
print(ave_abs, std_abs)

savename = "noise%d_N%d_3.png" % (NOISE, N)
mae = history.history['mae']
val_mae = history.history['val_mae']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(mae) + 1)

plt.subplot(2,1,1)
plt.plot(epochs, mae,  color="black", label = 'Training mae')
plt.plot(epochs, val_mae,  color="red", label = 'Valdation mae')
plt.legend()             # 凡例
plt.xlabel('Epochs')          # 軸ラベル
plt.ylabel('mae')
plt.ylim(0.0, 2.0)      # y軸の表示範囲
plt.subplot(2,1,2)
plt.plot(epochs, loss,  color="black", label = 'Training mae')
plt.plot(epochs, val_loss,  color="red", label = 'Valdation mae')
plt.legend()             # 凡例
plt.xlabel('Epochs')          # 軸ラベル
plt.ylabel('Loss')
plt.ylim(0.0, 2.0)
plt.show()
plt.savefig(savename) # png画像として出力
plt.close()

In [8]:
#CNN1の実装
import keras
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop
from keras.layers import Conv1D, MaxPooling1D

batch_size = 200  # 訓練データを200ずつのデータに分けて学習させる
epochs = 50 # 訓練データを繰り返し学習させる数

#データ形式の変更(シーケンス長, パラメータ数)
train_x = x_train.reshape(-1, 100, 1)
train_t = tx_train.T[1]
test_x = x_test.reshape(-1, 100, 1)
test_t = tx_test.T[1]

model = Sequential()
model.add(Conv1D(50, 3, padding='same', input_shape=(100, 1), activation='relu', kernel_initializer="he_normal"))
model.add(MaxPooling1D(2, padding='same'))
model.add(Conv1D(50, 3, padding='same', activation='relu', kernel_initializer="he_normal"))
model.add(MaxPooling1D(5, padding='same'))
model.add(Conv1D(10, 3, padding='same', activation='relu', kernel_initializer="he_normal"))
model.add(layers.Flatten())
model.add(Dense(100, activation='relu', kernel_initializer="he_normal"))
model.add(Dropout(0.2))
model.add(Dense(10, activation='relu', kernel_initializer="he_normal"))
model.add(Dropout(0.2))
model.add(Dense(1, activation='relu', kernel_initializer="he_normal"))

model.summary()

model.compile(loss='mse',
 optimizer='adam',
 metrics=['mae'])

callbacks = [keras.callbacks.TensorBoard(log_dir='my_log_dir_2', 
                                         histogram_freq=1)]

history = model.fit(train_x, train_t,
 batch_size=batch_size,
 epochs=epochs,
 verbose=1,
 validation_data=(test_x, test_t),
 callbacks=callbacks)

score = model.evaluate(test_x, test_t, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_16 (Conv1D)           (None, 100, 50)           200       
_________________________________________________________________
max_pooling1d_11 (MaxPooling (None, 50, 50)            0         
_________________________________________________________________
conv1d_17 (Conv1D)           (None, 50, 50)            7550      
_________________________________________________________________
max_pooling1d_12 (MaxPooling (None, 10, 50)            0         
_________________________________________________________________
conv1d_18 (Conv1D)           (None, 10, 10)            1510      
_________________________________________________________________
flatten_6 (Flatten)          (None, 100)               0         
_________________________________________________________________
dense_16 (Dense)             (None, 100)              

InvalidArgumentError: You must feed a value for placeholder tensor 'conv1d_1_input' with dtype float and shape [?,100,1]
	 [[node conv1d_1_input (defined at /Users/nagaiyuma/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:736) ]]

Original stack trace for 'conv1d_1_input':
  File "/Users/nagaiyuma/anaconda3/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/nagaiyuma/anaconda3/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/nagaiyuma/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/nagaiyuma/anaconda3/lib/python3.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/nagaiyuma/anaconda3/lib/python3.7/site-packages/ipykernel/kernelapp.py", line 505, in start
    self.io_loop.start()
  File "/Users/nagaiyuma/anaconda3/lib/python3.7/site-packa

In [ ]:
#modelの保存
save_model_path = "/Users/nagaiyuma/Desktop/maindata/model_cnn_1.h5"
model.save(save_model_path)

In [ ]:
#学習パラメータの取得
model = load_model(save_model_path)

In [ ]:
abs_error = abs(np.array(model.predict(test_x).flatten()- test_t))
ave_abs = np.average(abs_error)
std_abs = np.std(abs_error)
print(ave_abs, std_abs)


savename = "noise%d_N%d_1.png" % (NOISE, N)
mae = history.history['mae']
val_mae = history.history['val_mae']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(mae) + 1)

plt.subplot(2,1,1)
plt.plot(epochs, mae,  color="black", label = 'Training mae')
plt.plot(epochs, val_mae,  color="red", label = 'Valdation mae')
plt.legend()             # 凡例
plt.xlabel('Epochs')          # 軸ラベル
plt.ylabel('mae')
plt.ylim(0.0, 2.0)      # y軸の表示範囲
plt.subplot(2,1,2)
plt.plot(epochs, loss,  color="black", label = 'Training mae')
plt.plot(epochs, val_loss,  color="red", label = 'Valdation mae')
plt.legend()             # 凡例
plt.xlabel('Epochs')          # 軸ラベル
plt.ylabel('Loss')
plt.ylim(0.0, 2.0)
plt.show()
plt.savefig(savename) # png画像として出力
plt.close()